In [50]:
# check the affect of HEM issue on signal yield in run 2
# check the signal yield before and after removal of jets in problematic region
import ROOT as rt
import csv
import re
import sys
import collections
import os

from collections import OrderedDict
import uproot
import pandas as pd

import scipy
import awkward
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
sys.path.append('/storage/af/user/christiw/gpu/christiw/llp/delayed_jet_analyzer/lib/')
sys.path.append('/storage/af/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/python/')
from helper import make_datacard_2sig, make_datacard_2tag, weight_calc
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot
from helper_functions import deltaR, deltaPhi

import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 0


print(sys.version)

3.6.8 (default, Aug  7 2019, 17:28:10) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-39)]


# Load ntuples

In [51]:
fpath =OrderedDict()
tree = OrderedDict()
mass = [15, 40, 55]


old_ctau = {
#     'STodd_ms3p0':[100,500],
#     'SToEE_ms0p4':[10,50],
#     'SToGammaGamma_ms0p4':[10,50],
#     'SToKPlusKMinus_ms1p5':[38,187],
#     'SToK0K0_ms1p5':[38,187],
#     'SToPi0Pi0_ms0p4':[10,50],
#     'SToPi0Pi0_ms1p0':[25,125],
#     'SToPiPlusPiMinus_ms0p4':[10,50],
#     'SToPiPlusPiMinus_ms1p0':[25,125],
#     'STodd_ms7':[100, 1000, 10000, 100000],
#     'STodd_ms15':[100, 1000, 10000, 100000],
#     'STodd_ms40':[100, 1000, 10000, 100000],
#     'STodd_ms55':[100, 1000, 10000, 100000],
    
    'STodd_ms7':[1000],
    'STodd_ms15':[1000],
    'STodd_ms40':[1000],
    'STodd_ms55':[1000],
#     'SToTauTau_ms7':[100, 1000, 10000, 100000],
#     'SToTauTau_ms15':[100, 1000, 10000, 100000],
#     'SToTauTau_ms40':[100, 1000, 10000, 100000],
#     'SToTauTau_ms55':[100, 1000, 10000, 100000],
#     'SToBB_ms15':[100, 1000, 10000, 100000],
#     'SToBB_ms40':[100, 1000, 10000, 100000],
#     'SToBB_ms55':[100, 1000, 10000, 100000],
#     'SToEE_ms4p0':[100,500],
#     'SToGammaGamma_ms4p0':[100,500],
#     'SToPiPlusPiMinus_ms4p0':[100,500],
    
}

# prod = ['ggH', 'VBFH','ZH', 'WH', 'ttH','ggZH']
prod=['ggH']

ntupler_version = 'V1p17/'
mc_path = {}
analyzer_version = 'v1/v168/'
mc_central_path = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_Fall18/'+analyzer_version+'/normalized/'


for p in prod:
#     if 'WH' in p:continue
    for k in old_ctau.keys():
        for ct in old_ctau[k]:
            key = 'MC_'+p+'_'+k + '_'+str(ct)
            if (p == 'ggH' or p == 'VBFH') and ('ms7' in k or 'ms15' in k or 'ms40' in k or 'ms55' in k) and not (p =='VBFH' and 'dd' in k):
                mass = k[k.find('ms')+2:]
                if 'dd' in k and p == 'ggH':fpath[key] = mc_central_path+p+'_HToSSTodddd_MH-125_MS-'+mass+'_ctau-'+str(ct)+'_TuneCP5_13TeV-powheg-pythia8_59740pb_weighted.root'
            
            if not os.path.exists(fpath[key]):print(key, fpath[key])
            

            
# mass = [15, 40, 55]
# if not decay == 'bbbb': mass = [7, 15, 40, 55]

# OLD_CTAU = np.array([100, 1000, 10000, 100000])#in mm

# ntupler_version = 'V1p17/'


                     
NEvents = {}
# NEvents_genweight = {}
for k,v in fpath.items():
    root_dir = uproot.open(v) 
    if not root_dir: 
        print(k, "zombie")
        continue
    tree[k] = root_dir['MuonSystem']
#     NEvents[k] = root_dir['NEvents'][1]
    NEvents[k] = root_dir['NEvents']._fEntries
#     print(k)
#     if not 'data' in k: 
#         print(k, root_dir['NEvents']._fEntries)


root_dir = uproot.open('/storage/af/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/data/HiggsPtWeights/ZHToggZH_HiggsPtReweight.root') 
h_reweight = root_dir['higgsPthiggsEta']

# nCsc with different hit vetoing

In [49]:

for category in [0,1,2]:
#     if not category == 0:continue
    signal_yield = {}
    signal_unc = {}
    for k in tree.keys():
        signal_yield[k] = []
        signal_unc[k] = []
    for removeJetsinHEM in [0,1]:
        print('category {}, remove jets in HEM {}'.format(category, removeJetsinHEM))
        weight = {}
        nhits1 = {}
        nhits2 = {}
        sel_ev = {}
        cond = {}
        ggZH_weight = {}
        higgsEta = {}
        higgsPt = {}
        EE_prefiring = {}
        cluster_index = ''
        addNoiseFlag = 1
        # 0: 2 CSC; 1: 2DT; 2: csc+dt


        for k in list(tree.keys()):
        #     if  'data' in k:continue
        ########### SELECTION: CLUSTERS ############
            if 'data' in k: T = tree['data']
            else: T = tree[k]

            sel_csccluster = T.array('cscRechitCluster' + cluster_index + 'TimeSpreadWeightedAll')<20
            sel_csccluster = np.logical_and(sel_csccluster, T.array('cscRechitCluster' + cluster_index + 'Me11Ratio')<1)
            sel_csccluster = np.logical_and(sel_csccluster, np.logical_not(np.logical_and(T.array('cscRechitClusterMuonVetoPt') >= 30, T.array('cscRechitClusterMuonVetoGlobal'))))

            if 'data' in k: 
                sel_csccluster = np.logical_and(sel_csccluster, np.logical_and(T.array('cscRechitCluster' + cluster_index + 'TimeWeighted')< 12.5, \
                                                                                 T.array('cscRechitCluster' + cluster_index + 'TimeWeighted') > -5))
            else: 
                sel_csccluster = np.logical_and(sel_csccluster, np.logical_and(T.array('cscRechitCluster' + cluster_index + 'TimeWeighted')+0.66 < 12.5, \
                                                                                 T.array('cscRechitCluster' + cluster_index + 'TimeWeighted')+0.66 > -5))
            if removeJetsinHEM:
                hem_jetveto = np.logical_and(T.array('cscRechitCluster' + cluster_index + 'JetVetoEta') > -3, T.array('cscRechitCluster' + cluster_index + 'JetVetoEta') < -1.3)
                hem_jetveto = np.logical_and(hem_jetveto, T.array('cscRechitCluster' + cluster_index + 'JetVetoPhi') < -0.87)
                hem_jetveto = np.logical_and(hem_jetveto,T.array('cscRechitCluster' + cluster_index + 'JetVetoPhi') > -1.57)
                hem_jetveto = np.logical_or(hem_jetveto, T.array('cscRechitCluster' + cluster_index + 'JetVetoPt') < 30)
                sel_csccluster = np.logical_and( sel_csccluster, hem_jetveto)
                sel_csccluster = np.logical_and(sel_csccluster, np.abs(T.array('cscRechitCluster' + cluster_index + 'MetHEM_dPhi'))<1.2)
            else:
                sel_csccluster = np.logical_and(sel_csccluster, T.array('cscRechitCluster' + cluster_index + 'JetVetoPt')<30)
                sel_csccluster = np.logical_and(sel_csccluster, np.abs(T.array('cscRechitCluster' + cluster_index + 'Met_dPhi'))<1.2)


            sel_dtcluster = np.abs(T.array('dtRechitClusterMetEENoise_dPhi')) < 10000
            sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_not(np.logical_and(T.array('dtRechitClusterMuonVetoPt') >= 10, T.array('dtRechitClusterMuonVetoLooseId'))))
            sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_not(np.logical_and(T.array('dtRechitClusterMaxStation')==1, T.array('dtRechitClusterMaxStationRatio')>0.9)))


            cut = 5
            station = (T.array('dtRechitClusterNSegmentStation1')>cut).astype(int)+(T.array('dtRechitClusterNSegmentStation2')>cut).astype(int)\
        +(T.array('dtRechitClusterNSegmentStation3')>cut).astype(int)+(T.array('dtRechitClusterNSegmentStation4')>cut).astype(int)

            max_station = np.maximum(np.maximum(np.maximum(T.array('dtRechitClusterNSegmentStation1'), T.array('dtRechitClusterNSegmentStation2')), T.array('dtRechitClusterNSegmentStation3')), T.array('dtRechitClusterNSegmentStation4'))
            min_station = np.minimum(np.minimum(np.minimum(T.array('dtRechitClusterNSegmentStation1'), T.array('dtRechitClusterNSegmentStation2')), T.array('dtRechitClusterNSegmentStation3')), T.array('dtRechitClusterNSegmentStation4'))

            sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_or(station<4, min_station/max_station<0.4)) #remove if both clusters are 4 stations
            if addNoiseFlag and not 'data' in k: 
                sel_dtcluster = np.logical_and(sel_dtcluster, (T.array('dtRechitClusterSize')+T.array('dtRechitClusterNoiseHit')) >= 50)
            else: sel_dtcluster = np.logical_and(sel_dtcluster, T.array('dtRechitClusterSize') >= 50)

            # cosmic muon veto 
            sel_cosmic = np.logical_and(T.array('dtRechitClusterNOppositeSegStation1')>0, T.array('dtRechitClusterNOppositeSegStation2')>0)
            sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation3')>0)
            sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation4')>0)
            sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation1')+T.array('dtRechitClusterNOppositeSegStation2')+\
                                       T.array('dtRechitClusterNOppositeSegStation3')+T.array('dtRechitClusterNOppositeSegStation4')>=6)
            nstation = (T.array('dtRechitClusterNSegmentStation1')>1).astype(int)+(T.array('dtRechitClusterNSegmentStation2')>1).astype(int)\
            +(T.array('dtRechitClusterNSegmentStation3')>1).astype(int)+(T.array('dtRechitClusterNSegmentStation4')>1).astype(int)

            sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_not(np.logical_and(nstation>=3, sel_cosmic)))

            if removeJetsinHEM:
                hem_jetveto = np.logical_and(T.array('dtRechitCluster' + cluster_index + 'JetVetoEta') > -3, T.array('dtRechitCluster' + cluster_index + 'JetVetoEta') < -1.3)
                hem_jetveto = np.logical_and(hem_jetveto, T.array('dtRechitCluster' + cluster_index + 'JetVetoPhi') < -0.87)
                hem_jetveto = np.logical_and(hem_jetveto,T.array('dtRechitCluster' + cluster_index + 'JetVetoPhi') > -1.57)
                hem_jetveto = np.logical_or(hem_jetveto, T.array('dtRechitCluster' + cluster_index + 'JetVetoPt') < 50)
                sel_dtcluster = np.logical_and( sel_dtcluster, hem_jetveto)
                sel_dtcluster = np.logical_and(sel_dtcluster, np.abs(T.array('dtRechitCluster' + cluster_index + 'MetHEM_dPhi'))<1)

            else:
                sel_dtcluster = np.logical_and(sel_dtcluster, np.abs(T.array('dtRechitClusterJetVetoPt')) < 50)
                sel_dtcluster = np.logical_and(sel_dtcluster, np.abs(T.array('dtRechitCluster' + cluster_index + 'Met_dPhi'))<1)
        ########### SELECTION: JETS ############

            sel_jet = np.logical_and(T.array('jetPt') > 30, np.abs(T.array('jetEta')) < 2.4 )
            sel_jet = np.logical_and(T.array('jetTightPassId'), sel_jet)



        ########### SELECTION: NOISE IN DT ############

            spike = np.logical_and( T.array('nDTRechitsSector')[:,0,0,7]>50,  T.array('nDTRechitsSector')[:,0,0,7]+T.array('nDTRechitsSector')[:,0,0,8]+T.array('nDTRechitsSector')[:,0,0,9]>120)
            spike = np.logical_and(spike, T.array('nDTRechitsSector')[:,0,0,8]>25)
            spike = np.logical_and(spike, T.array('nDTRechitsSector')[:,0,0,9]>10)



        ########### SELECTION: EVENTS ############

            sel_ev[k] = T.array('METNoMuTrigger')
#             if removeJetsinHEM:sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('metHEM') >= 200)
#             else:sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('met') >= 200)
            sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('met') >= 200)
            sel_ev[k] = np.logical_and(sel_ev[k] , sel_jet.sum()>=1)
            sel_ev[k] = np.logical_and(sel_ev[k], (T.array('nDtRings')+T.array('nCscRings'))<10)
            sel_ev[k] = np.logical_and(sel_ev[k],T.array('Flag2_all'))
            sel_ev[k] = np.logical_and(sel_ev[k] , np.logical_not(spike))

        ########### BRANCHES ############




            if category == 0:
                sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum()== 2)
                sel_ev[k]  = np.logical_and(sel_ev[k],sel_dtcluster.sum()== 0)
                cond[k] = deltaR(T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,0], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,0],\
                                T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,1], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,1])<2


                nhits1[k] =  T.array('cscRechitCluster' + cluster_index + 'Size')[sel_csccluster][sel_ev[k]][cond[k]][:,0]
                nhits2[k] =  T.array('cscRechitCluster' + cluster_index + 'Size')[sel_csccluster][sel_ev[k]][cond[k]][:,1]
        #         deltaRCluster[k] = deltaR(T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,0], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,0],\
        #                         T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,1], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,1])
            elif category == 1:
                sel_ev[k]  = np.logical_and(sel_ev[k],sel_dtcluster.sum()== 2)
                sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum()== 0)
                cond[k] = np.abs(T.array('dtRechitCluster' + cluster_index + 'Eta')[sel_dtcluster][sel_ev[k]][:,0]-\
                                                            T.array('dtRechitCluster' + cluster_index + 'Eta')[sel_dtcluster][sel_ev[k]][:,1])>=0
                if addNoiseFlag and not 'data' in k:
                    nhits1[k] =  (T.array('dtRechitClusterNoiseHit')+T.array('dtRechitClusterSize'))[sel_dtcluster][sel_ev[k]][:,0]
                    nhits2[k] =  (T.array('dtRechitClusterNoiseHit')+T.array('dtRechitClusterSize'))[sel_dtcluster][sel_ev[k]][:,1]
                else:
                    nhits1[k] =  T.array('dtRechitClusterSize')[sel_dtcluster][sel_ev[k]][:,0]
                    nhits2[k] =  T.array('dtRechitClusterSize')[sel_dtcluster][sel_ev[k]][:,1]
            elif category == 2:
                sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum() == 1)
                sel_ev[k]  = np.logical_and(sel_ev[k],sel_dtcluster.sum() == 1)

        #         cond[k] =  np.abs(T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,0]-\
        #                                                     T.array('dtRechitCluster' + cluster_index + 'Eta')[sel_dtcluster][sel_ev[k]][:,0])<2.0

                cond[k] = deltaR(T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,0], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,0],\
                                T.array('dtRechitCluster' + cluster_index + 'Eta')[sel_dtcluster][sel_ev[k]][:,0], T.array('dtRechitCluster' + cluster_index + 'Phi')[sel_dtcluster][sel_ev[k]][:,0])<2.5
                if addNoiseFlag and not 'data' in k:
                    nhits1[k] =  (T.array('dtRechitClusterNoiseHit')+T.array('dtRechitClusterSize'))[sel_dtcluster][sel_ev[k]][cond[k]][:,0]
                else:
                    nhits1[k] =  T.array('dtRechitClusterSize')[sel_dtcluster][sel_ev[k]][cond[k]][:,0]
                nhits2[k] =  T.array('cscRechitCluster' + cluster_index + 'Size')[sel_csccluster][sel_ev[k]][cond[k]][:,0]



            else:
                assert(False)


            higgsPt[k] = T.array('gHiggsPt')[sel_ev[k]][cond[k]]
            higgsEta[k] = T.array('gHiggsEta')[sel_ev[k]][cond[k]]

            ggZH_weight[k]=h_reweight.values[np.argmax(h_reweight.edges[0]>higgsPt[k][:,None],axis=1)-1, np.argmax(h_reweight.edges[1]>np.abs(higgsEta[k])[:,None],axis=1)-1]


            if 'ggH' in k: weight[k] = (T.array('weight')*T.array('pileupWeight')*T.array('higgsPtWeight')*T.array('metSF'))[sel_ev[k]][cond[k]]
            else:weight[k] = (T.array('weight')*T.array('pileupWeight')*T.array('metSF'))[sel_ev[k]][cond[k]]
            if 'ggZH' in k: weight[k] *= ggZH_weight[k]
#             print(k,'\t', np.sum(weight[k]))
            signal_yield[k].append(np.sum(weight[k]))
            signal_unc[k].append(np.sum(weight[k]**2)**0.5)

    for k in signal_yield.keys():
        print(k, '\t', round(signal_yield[k][0],2), '+-',round(signal_unc[k][0],2),'\t',round(signal_yield[k][1],2), '+-',round(signal_unc[k][1],2), '\t',signal_yield[k][1]/ signal_yield[k][0]-1)

category 0, remove jets in HEM 0
category 0, remove jets in HEM 1
MC_ggH_STodd_ms7_1000 	 42.7 +- 7.29 	 41.84 +- 7.24 	 -0.020201385021209717
MC_ggH_STodd_ms15_1000 	 92.31 +- 10.34 	 91.47 +- 10.3 	 -0.0090714693069458
MC_ggH_STodd_ms40_1000 	 170.86 +- 14.19 	 167.27 +- 14.03 	 -0.021000385284423828
MC_ggH_STodd_ms55_1000 	 12.77 +- 3.8 	 12.77 +- 3.8 	 0.0
category 1, remove jets in HEM 0
category 1, remove jets in HEM 1
MC_ggH_STodd_ms7_1000 	 224.11 +- 16.22 	 220.13 +- 16.08 	 -0.01773935556411743
MC_ggH_STodd_ms15_1000 	 429.07 +- 22.89 	 421.86 +- 22.66 	 -0.016796529293060303
MC_ggH_STodd_ms40_1000 	 439.48 +- 22.59 	 436.69 +- 22.52 	 -0.006332755088806152
MC_ggH_STodd_ms55_1000 	 31.38 +- 6.0 	 30.19 +- 5.88 	 -0.037999629974365234
category 2, remove jets in HEM 0
category 2, remove jets in HEM 1
MC_ggH_STodd_ms7_1000 	 340.99 +- 20.66 	 337.51 +- 20.58 	 -0.010214745998382568
MC_ggH_STodd_ms15_1000 	 681.48 +- 28.37 	 671.44 +- 28.13 	 -0.014730513095855713
MC_ggH_STodd_ms